# MeshCat Python

In [1]:
import numpy as np
import os
import time

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [2]:
# Create a new visualizer
vis = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


By default, creating the `Visualizer` will start up a meshcat server for you in the background. The easiest way to open the visualizer is with its ``open`` method:

In [3]:
vis.open()

<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x0000026968BB8FD0> at path: <meshcat.path.Path object at 0x0000026968BB8370>>

If ``vis.open()`` does not work for you, you can also point your browser to the server's URL:

In [4]:
vis.url()

'http://127.0.0.1:7000/static/'

To create a 3D object, we use the `set_object` method:

In [5]:
vis.set_object(g.Box([0.2, 0.2, 0.2]))

And to move that object around, we use `set_transform`:

In [6]:
for theta in np.linspace(0, 2 * np.pi, 200):
    vis.set_transform(tf.rotation_matrix(theta, [0, 0, 1]))
    time.sleep(0.005)

MeshCat also supports embedding a 3D view inside a Jupyter notebook cell:

In [7]:
vis.jupyter_cell()

Notice how the 3D scene displayed in the Jupyter cell matches the one in the external window. The meshcat server process remembers the objects and transforms you've sent, so opening a new browser pointing to the same URL should give you the same scene. 

Calling `set_object` again will replace the existing Box:

In [8]:
vis.set_object(g.Box([0.1, 0.1, 0.2]))

We can also delete the box:

In [9]:
vis.delete()

## The Scene Tree

Obviously, we will often want to draw more than one object. So how do we do that? The fundamental idea of MeshCat is that it gives direct access to the *scene graph*. You can think of the scene as a tree of objects, and we name each object in the tree by its *path* from the root of the tree. Children in the tree inherit the transformations applied to their parents. So, for example, we might have a `robot` at the path `/robot`, and that robot might have a child called `head` at the path `/robot/head`. Each path in the tree can have a different geometry associated.

First, let's create the robot. We access paths in the tree by indexing into the Visualizer:

In [10]:
vis["robot"].set_object(g.Box([0.15, 0.35, 0.4]))

Now let's give the robot a head:

In [11]:
vis["robot"]["head"].set_object(g.Box([0.2, 0.2, 0.2]))
vis["robot"]["head"].set_transform(tf.translation_matrix([0, 0, 0.32]))

We can move the entire robot by setting the transform of the `/robot` path:

In [12]:
for x in np.linspace(0, np.pi, 100):
    vis["robot"].set_transform(tf.translation_matrix([np.sin(x), 0, 0]))
    time.sleep(0.01)

And we can move just the head by setting the transform of `/robot/head`:

In [13]:
for x in np.linspace(0, 2 * np.pi, 100):
    # vis["robot/head"] is a shorthand for vis["robot"]["head"]
    vis["robot/head"].set_transform(
        tf.translation_matrix([0, 0, 0.32]).dot(
            tf.rotation_matrix(x, [0, 0, 1])))
    time.sleep(0.01)

We can delete the head...

In [14]:
vis["robot/head"].delete()

...or the entire robot:

In [15]:
vis["robot"].delete()

## Other Geometries

MeshCat supports several geometric primitives as well as meshes (represented by `.obj`, `.dae`, or `.stl` files). You can also specify a material to describe the object's color, reflectivity, or texture:

In [16]:
vis["sphere"].set_object(g.Sphere(0.1), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))

In [17]:
vis["sphere"].delete()

MeshCat can load `.obj`, `.dae`, and `.stl` meshes via the `ObjMeshGeometry`, `DaeMeshGeometry`, and `StlMeshGeometry` types respectively:

In [21]:
vis["robots/valkyrie/head"].set_object(
    g.ObjMeshGeometry.from_file(
        os.path.join(meshcat.viewer_assets_path(), "data/head_multisense.obj")),
    g.MeshLambertMaterial(
        map=g.ImageTexture(
            image=g.PngImage.from_file(
                os.path.join(meshcat.viewer_assets_path(), "data/HeadTextureMultisense.png"))
        )
    )
)

The `PointCloud()` function is a helper to create a `Points` object with a `PointsGeometry` and `PointsMaterial`:

In [22]:
verts = np.random.rand(3, 100000)
vis["perception/pointclouds/random"].set_object(
    g.PointCloud(position=verts, color=verts))
vis["perception/pointclouds/random"].set_transform(
    tf.translation_matrix([0, 1, 0]))

In [20]:
vis["robots"].delete()
vis["perception"].delete()

## Cart-Pole

Here's a simple example of visualizing a mechanism:

In [ ]:
cart_pole = vis["cart_pole"]
cart_pole.delete()
cart = cart_pole["cart"]
pivot = cart["pivot"]
pole = pivot["pole"]
cart.set_object(g.Box([0.5, 0.3, 0.2]))
pole.set_object(g.Box([1, 0.05, 0.05]))
pole.set_transform(tf.translation_matrix([0.5, 0, 0]))
pivot.set_transform(tf.rotation_matrix(-np.pi/2, [0, 1, 0]))

In [ ]:
for x in np.linspace(-np.pi, np.pi, 200):
    cart.set_transform(tf.translation_matrix([np.sin(x), 0, 0]))
    pivot.set_transform(tf.rotation_matrix(x / 4 - np.pi / 2, [0, 1, 0]))
    time.sleep(0.01)